In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from sklearn.utils import shuffle

import warnings
warnings.filterwarnings("ignore")
pathDrive = 'drive/MyDrive/UNICT/_TESI/dati/'

df = pd.read_csv(pathDrive + 'dati_interpolati_diff_1d-2d_8gg - riordinati.csv', sep=';')

df['edam_ecne_eplu_diff'] = df['edam_ecne_eplu_diff'].str.replace(',','.').astype(float)#
df['edam_ecor_emcn_diff'] = df['edam_ecor_emcn_diff'].str.replace(',','.').astype(float)
df['edam_emsg_eplu_diff'] = df['edam_emsg_eplu_diff'].str.replace(',','.').astype(float)
df['edam_epdn_ecne_diff'] = df['edam_epdn_ecne_diff'].str.replace(',','.').astype(float)
df['edam_epdn_emcn_diff'] = df['edam_epdn_emcn_diff'].str.replace(',','.').astype(float)
df['eint_ecpn_emgl_diff'] = df['eint_ecpn_emgl_diff'].str.replace(',','.').astype(float)#
df['eint_esln_emgl_diff'] = df['eint_esln_emgl_diff'].str.replace(',','.').astype(float)
df['eint_esln_espc_diff'] = df['eint_esln_espc_diff'].str.replace(',','.').astype(float)
df['emeg_ecpn_emgl_diff'] = df['emeg_ecpn_emgl_diff'].str.replace(',','.').astype(float)
df['emfn_ecor_ecri_diff'] = df['emfn_ecor_ecri_diff'].str.replace(',','.').astype(float)
df['emfn_ecor_emcn_diff'] = df['emfn_ecor_emcn_diff'].str.replace(',','.').astype(float)
df['emfn_emcn_epdn_diff'] = df['emfn_emcn_epdn_diff'].str.replace(',','.').astype(float)
df['epdn_ecne_ecpn_diff'] = df['epdn_ecne_ecpn_diff'].str.replace(',','.').astype(float)
df['epdn_eint_ecpn_diff'] = df['epdn_eint_ecpn_diff'].str.replace(',','.').astype(float)##
df['epdn_eint_espc_diff'] = df['epdn_eint_espc_diff'].str.replace(',','.').astype(float)
df['epdn_emfn_espc_diff'] = df['epdn_emfn_espc_diff'].str.replace(',','.').astype(float)
df['eplu_ecne_ecpn_diff'] = df['eplu_ecne_ecpn_diff'].str.replace(',','.').astype(float)
df['eplu_emeg_ecpn_diff'] = df['eplu_emeg_ecpn_diff'].str.replace(',','.').astype(float)
df['eplu_emeg_emal_diff'] = df['eplu_emeg_emal_diff'].str.replace(',','.').astype(float)
df['eplu_emsg_emal_diff'] = df['eplu_emsg_emal_diff'].str.replace(',','.').astype(float)


df_train = df[:126]
'''
df_val = df[126:170]
df_test = df[170:]
'''
df_test = df[126:]

y_train = df_train['evento'].astype(int).values
#y_val = df_val['evento'].astype(int).values
y_test = df_test['evento'].astype(int).values
#_dftarget = df['evento'].astype(int).values

In [3]:
def toDataFrame(df):
  format = '%d/%m/%Y'
  df['date'] = pd.to_datetime(df['day'], format=format)
  df = df.set_index('date')
  df = df.drop(columns=['day'])
  df = df.drop(columns=['evento'])

  for i in reversed(range(len(df.columns))):
    if df.dtypes[df.columns[i]] == 'object':
      df = df.drop(df.columns[i], axis=1)

  return df

df = toDataFrame(df)
df_train = toDataFrame(df_train)
#df_val = toDataFrame(df_val)
df_test = toDataFrame(df_test)

############## normalizzo tutto il dataframe
#normalized_df = (df - df.mean()) / df.std()

############## normalizzo le tre parti del dataset in base alla varianza del training
df_train_mean = df_train.mean()
df_train_std = df_train.std()

normalized_df_train = (df_train - df_train_mean) / df_train_std
#normalized_df_val   = (df_val - df_train_mean) / df_train_std
normalized_df_test  = (df_test - df_train_mean) / df_train_std

In [4]:
from sklearn import datasets, decomposition

pca_feature_number = 20
pca = decomposition.PCA(n_components = pca_feature_number)
pca.fit(normalized_df_train) #le PCA si genera a partire dai soli dati del training

def applicaPca(pca_model, normalized_df):
  temp_df = normalized_df.copy()
  new_df = pca_model.transform(temp_df)

  for i in range(pca_model.n_components):
    temp_df['f_' + str(i)] = new_df[:,i]

  return temp_df.filter(like='f_')

df_train_pca = applicaPca(pca, normalized_df_train)
#df_val_pca = applicaPca(pca, normalized_df_val)
df_test_pca = applicaPca(pca, normalized_df_test)

#df_test_pca.info()
#pca.explained_variance_ratio_.cumsum()

In [ ]:
'''
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier

#calcola lo score in base all'algoritmo scelto (Random Forest, Logistic RegressionCV, ..) di tutte le prime componenti principali
for i in range(pca_feature_number):
  #clf = LogisticRegressionCV(cv=5, random_state=0, class_weight='balanced') #scoring per cambiare metrica
  clf = RandomForestClassifier(max_depth=3, random_state=0)

  clf.fit(df_train_pca.values[:,:i+1], y_train)

  acc_train = clf.score(df_train_pca.values[:,:i+1],  y_train)*100
  #acc_val   = clf.score(df_val_pca.values[:,:i+1],    y_val)*100
  acc_test  = clf.score(df_test_pca.values[:,:i+1],   y_test)*100
  print( str(i+1) + ";" + "{:.1f}".format(acc_train) + ";" + "{:.1f}".format(acc_test) )
'''

In [ ]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import cross_val_score
import numpy as np

#y = _dftarget
#split_at = 150

#max_f1_score = 0
#max_f1_score_who = ""
max_accuracy_train = 0
#max_accuracy_val = 0
max_accuracy_tot = 0
max_accuracy_test = 0

max_accuracy_train_who = ""
#max_accuracy_val_who = ""
max_accuracy_tot_who = ""
max_accuracy_test_who = ""

stato_iniziale = True
stato_iniziale_sequenza = [6,7,9,11,12,16,17,18,19]

def get_range(indice):
  if stato_iniziale:
    return range(stato_iniziale_sequenza[indice], pca_feature_number)
  else:
    return range(pca_feature_number)


#pca_feature_number = 20 #SOLO SE NON SI USA LA PCA

for i in get_range(0):
  for j in get_range(1):
    for a in get_range(2):
      for b in get_range(3):
        for c in get_range(4):
          for d in get_range(5):
            for e in get_range(6):
              for f in get_range(7):
                for g in get_range(8):

                  stato_iniziale = False
                  if i<j and j<a and a<b and b<c and c<d and d<e and e<f and f<g:

                    who = "area " + str(i) + " - " + str(j) + " - " + str(a) + " - " + str(b) + " - " + str(c) + " - " + str(d) + " - " + str(e) + " - " + str(f) + " - " + str(g)

                    X_train = df_train_pca.values[:, [i,j,a,b,c,d,e,f,g]]
                    #X_val   = df_val_pca.values[:, [i,j,a,b,c,d,e,f]]
                    X_test  = df_test_pca.values[:, [i,j,a,b,c,d,e,f,g]]

                    #X_train = X[:split_at]
                    ##X_test = X[split_at:]

                    #y_train = y[:split_at]
                    #y_test = y[split_at:]

                    ################ LOGISTIC REGRESSION with CROSS VALIDATION
                    clf = LogisticRegressionCV(cv=5, random_state=0, class_weight='balanced') #scoring per cambiare metrica
                    ################ RANDOM FOREST
                    #clf = RandomForestClassifier(max_depth=5, random_state=0)

                    clf.fit(X_train, y_train)
                    acc_train = clf.score(X_train,  y_train,  sample_weight=None)*100
                    #acc_val   = clf.score(X_val,    y_val,    sample_weight=None)*100
                    acc_test  = clf.score(X_test,   y_test,    sample_weight=None)*100
                    #acc_s = np.mean(cross_val_score(clf, X_train, y_train, cv=5))*100

                    acc_tot = acc_train + acc_test

                    if (max_accuracy_train < acc_train or max_accuracy_test < acc_test or max_accuracy_tot < acc_tot) or (acc_test > 80) :
                      print( who + ";" + "{:.1f}".format(acc_train) + ";" + "{:.1f}".format(acc_test) )

                      if max_accuracy_train < acc_train:
                        max_accuracy_train = acc_train
                        max_accuracy_train_who = who

                      if max_accuracy_test < acc_test:
                        max_accuracy_test = acc_test
                        max_accuracy_test_who = who

                      if max_accuracy_tot < acc_tot:
                        max_accuracy_tot = acc_tot


#print( max_f1_score_who )
#print( max_accuracy_score_who )
print( "---------------------> " + max_accuracy_train_who + " " + str(max_accuracy_train) )
print( "---------------------> " + max_accuracy_test_who + " " + str(max_accuracy_test) )
#print( conf_mat )

29116; area 0 - 1 - 6 - 7 - 11 - 14 - 15 - 16 - 19;58.7;81.0
29117; area 0 - 1 - 6 - 7 - 11 - 14 - 15 - 17 - 18;60.3;63.5
29121; area 0 - 1 - 6 - 7 - 11 - 14 - 16 - 17 - 19;62.7;69.8
29124; area 0 - 1 - 6 - 7 - 11 - 15 - 16 - 17 - 18;63.5;71.4
29125; area 0 - 1 - 6 - 7 - 11 - 15 - 16 - 17 - 19;64.3;69.8
29205; area 0 - 1 - 6 - 8 - 9 - 10 - 11 - 15 - 18;65.1;57.1
29285; area 0 - 1 - 6 - 8 - 9 - 11 - 12 - 16 - 18;65.9;71.4
29437; area 0 - 1 - 6 - 8 - 10 - 11 - 14 - 16 - 19;67.5;65.1
29581; area 0 - 1 - 6 - 8 - 11 - 14 - 15 - 18 - 19;61.1;79.4
31892; area 0 - 2 - 3 - 4 - 5 - 6 - 8 - 9 - 11;69.8;57.1
31911; area 0 - 2 - 3 - 4 - 5 - 6 - 8 - 11 - 13;70.6;55.6
32478; area 0 - 2 - 3 - 4 - 5 - 8 - 14 - 15 - 18;71.4;50.8
34287; area 0 - 2 - 3 - 4 - 8 - 11 - 15 - 16 - 18;72.2;60.3
35124; area 0 - 2 - 3 - 5 - 6 - 8 - 10 - 16 - 18;69.8;71.4
35483; area 0 - 2 - 3 - 5 - 6 - 12 - 13 - 16 - 18;70.6;71.4
35550; area 0 - 2 - 3 - 5 - 7 - 8 - 9 - 10 - 18;73.0;54.0
45253; area 0 - 2 - 5 - 6 - 7 - 11 - 15 - 